In [9]:
import numpy as np
import tqdm

import atm
from atm.simclr import resnet

In [7]:
from PIL import Image
from typing import Any, Callable, Optional, Tuple
from torchvision.datasets.vision import VisionDataset
from functools import partial
from astrobf.tmo import Mantiuk_Seidel

from atm.loader import TonemapImageDataset

In [16]:
import argparse 

args = argparse.Namespace()

args.data='./datasets' 
args.dataset_name='cifar10'
args.arch='resnet50'
args.workers=1
args.epochs=300 
args.batch_size=256 
args.lr=0.02
args.wd=0.0005
args.disable_cuda=False
args.fp16_precision=True
args.out_dim=128
args.log_every_n_steps=100
args.temperature=0.07
args.n_views = 2
args.gpu_index=0
args.device='cuda' if torch.cuda.is_available() else 'cpu'

print("Using device:", args.device)

Using device: cuda


In [10]:
import pickle

ddir = "../../tonemap/bf_data/Nair_and_Abraham_2010/"

fn = ddir + "all_gals_recentered.pickle"
all_gals = pickle.load(open(fn, "rb"))

all_gals = all_gals[1:] # Why the first galaxy image is NaN?

good_gids = np.array([gal['img_name'] for gal in all_gals])

from astrobf.utils.misc import load_Nair
cat_data = load_Nair(ddir + "catalog/table2.dat")
# pd dataframe

cat = cat_data[cat_data['ID'].isin(good_gids)]

tmo_params = {'b': 6.0,  'c': 3.96, 'dl': 9.22, 'dh': 2.45}

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
from torchvision.transforms import transforms

img_size = 64

test_transform = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.2])])

train_loader = TonemapImageDataset(all_gals, partial(Mantiuk_Seidel, **tmo_params),
                                     labels=cat['TT'].to_numpy(),
                                     train=False, 
                                     transform=test_transform)

In [17]:
model = resnet.resnet50(pretrained=False, num_classes=10, num_channels=1) 
# 10 classes: -5 ~ 6 

optimizer = torch.optim.Adam(model.parameters(), args.lr, weight_decay=args.wd)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                           last_epoch=-1)

NameError: name 'train_loader' is not defined

In [ ]:
for epoch_counter in range(self.args.epochs):
    # tqdm = progressbar generator
    for images, _ in tqdm(train_loader):
        images = torch.cat(images, dim=0)
        images = images.to(self.args.device)

        # autocast <- AMP
        with autocast(enabled=self.args.fp16_precision):
            features = self.model(images)
            ################################
            logits, labels = self.info_nce_loss(features)
            loss = self.criterion(logits, labels)
            ################################
        print("feature min max", 
                np.mean([ff.cpu().detach().min() for ff in features]),
                np.mean([ff.cpu().detach().max() for ff in features]))
        print("current loss", loss)


        self.optimizer.zero_grad()

        scaler.scale(loss).backward()

        scaler.step(self.optimizer)
        scaler.update()